In [1]:
import pandas as pd
import numpy as np
import math as m
from scipy.optimize import fsolve

In [2]:
# Original DataFrame
unitWeights = {
    'Depth[m]': [0, 12, 12.01, 36, 36.01, 52.5, 52.51, 60],
    'effUnitWeight[kN/m3]': [4, 4, 7, 7, 8, 8, 10, 10],
}
df = pd.DataFrame(unitWeights)

# Create a new depth range with 0.01 m increments
new_depths = np.arange(0, df['Depth[m]'].max() + 0.01, 0.01)

# Interpolate the unit weights
unitWeights_df = pd.DataFrame({'Depth[m]': new_depths})
unitWeights_df['effUnitWeight[kN/m3]'] = np.interp(
    new_depths, 
    df['Depth[m]'], 
    df['effUnitWeight[kN/m3]']
)

# Display the resulting DataFrame
print(unitWeights_df)


      Depth[m]  effUnitWeight[kN/m3]
0         0.00                   4.0
1         0.01                   4.0
2         0.02                   4.0
3         0.03                   4.0
4         0.04                   4.0
...        ...                   ...
5996     59.96                  10.0
5997     59.97                  10.0
5998     59.98                  10.0
5999     59.99                  10.0
6000     60.00                  10.0

[6001 rows x 2 columns]


In [3]:
# Create a deep copy
effSigma_df = unitWeights_df.copy(deep=True)


# Add a new column for vertical effective stress
effSigma_df['VerticalEffectiveStress[kPa]'] = 0.0

# Compute the vertical effective stress iteratively
for i in range(1, len(effSigma_df)):
    # Get the depth difference
    depth_diff = effSigma_df.loc[i, 'Depth[m]'] - effSigma_df.loc[i - 1, 'Depth[m]']
    # Compute the effective stress
    effSigma_df.loc[i, 'VerticalEffectiveStress[kPa]'] = (
        effSigma_df.loc[i - 1, 'VerticalEffectiveStress[kPa]'] +
        effSigma_df.loc[i, 'effUnitWeight[kN/m3]'] * depth_diff
    )

# Display the resulting DataFrame
print(effSigma_df.to_string())


      Depth[m]  effUnitWeight[kN/m3]  VerticalEffectiveStress[kPa]
0         0.00                   4.0                          0.00
1         0.01                   4.0                          0.04
2         0.02                   4.0                          0.08
3         0.03                   4.0                          0.12
4         0.04                   4.0                          0.16
5         0.05                   4.0                          0.20
6         0.06                   4.0                          0.24
7         0.07                   4.0                          0.28
8         0.08                   4.0                          0.32
9         0.09                   4.0                          0.36
10        0.10                   4.0                          0.40
11        0.11                   4.0                          0.44
12        0.12                   4.0                          0.48
13        0.13                   4.0                          

In [4]:
def compute_f_integral(target_depth, func, *args, **kwargs):
    df = func(*args, **kwargs)  # Apply the transformation function
    if 'Pu' not in df.columns or 'EquivalentDepth[m]' not in df.columns:
        raise ValueError("The input function must output a DataFrame with 'Pu' and 'EquivalentDepth[m]' columns.")
    df['F-integral'] = 0.0
    for i in range(1, len(df)):
        Pu_current = df.loc[i, 'Pu']
        Pu_previous = df.loc[i - 1, 'Pu']
        Depth_current = df.loc[i, 'EquivalentDepth[m]']
        Depth_previous = df.loc[i - 1, 'EquivalentDepth[m]']
        delta_F = 0.5 * (Pu_current + Pu_previous) * (Depth_current - Depth_previous)
        df.loc[i, 'F-integral'] = df.loc[i - 1, 'F-integral'] + delta_F
    f_integral_at_target = df.loc[df['Depth[m]'] == target_depth, 'F-integral'].values
    return f_integral_at_target[0] if len(f_integral_at_target) > 0 else None

def api_clay_pu(df, Su, D, J, d_adj=0):
    df = df.copy(deep=True)
    df['EquivalentDepth[m]'] = np.maximum(df['Depth[m]'] + d_adj, 0)
    df['Pu_1'] = (3 * Su + df['VerticalEffectiveStress[kPa]']) * D + J * Su * df['EquivalentDepth[m]']
    df['Pu_2'] = 9 * Su * D
    df['Pu'] = np.minimum(df['Pu_1'], df['Pu_2'] )
    return df

def api_sand_pu(df, phi, D, d_adj=0):
    df = df.copy(deep=True)
    # initial subgrade modulus in kpa from fit with API (2014)

    # Calculate Pmax (regular API)
    ## Factors according to Mosher and Dawkins 2008.(regular API)
    b = 0.4
    Beta = 45 + phi / 2
    rad = m.pi / 180
    C1 = (
        (b * m.tan(phi * rad) * m.sin(Beta * rad))
        / (m.tan((Beta - phi) * rad) * m.cos((phi / 2) * rad))
        + ((m.tan(Beta * rad)) ** 2 * m.tan((phi / 2) * rad)) / (m.tan((Beta - phi) * rad))
        + b * m.tan(Beta * rad) * (m.tan(phi * rad) * m.sin(Beta * rad) - m.tan((phi / 2) * rad))
    )
    C2 = m.tan(Beta * rad) / m.tan((Beta - phi) * rad) - (m.tan((45 - phi / 2) * rad)) ** 2
    C3 = b * m.tan(phi * rad) * (m.tan(Beta * rad)) ** 4 + (m.tan((45 - phi / 2) * rad)) ** 2 * (
        (m.tan(Beta * rad)) ** 8 - 1
    )

    df['EquivalentDepth[m]'] = np.maximum(df['Depth[m]'] + d_adj, 0)

    df['Pu_s'] = (C1 * df['VerticalEffectiveStress[kPa]'] * df['EquivalentDepth[m]']) + C2 * df['VerticalEffectiveStress[kPa]'] * D
    df['Pu_d'] = C3 * df['VerticalEffectiveStress[kPa]'] * D

    ## Pmax for shallow and deep zones (regular API)
    df['Pu'] = np.minimum(df['Pu_s'], df['Pu_d'])
    return df

def frankeRollins2013_pu(df, Sr, D, J, d_adj=0):
    if D > 2.6:
        p_d = 9.24
    elif D > 0.3:
        p_d = 3.81 * np.log(D) + 5.6
    else:
        p_d = (D / 0.3) * 3.81 * np.log(0.3) + 5.6

    df['z_depth'] = np.minimum(6.0, df['VerticalEffectiveStress[kPa]']/10.0)

    # derive static curve Rollins 2005
    df['A'] = (0.0000003) * ((df['z_depth'] + 1) ** 6.05)
    df['B'] = 2.80 * ((df['z_depth'] + 1) ** 0.11)
    df['C'] = 2.85 * ((df['z_depth'] + 1) ** -0.41)
    
    df['Pu_Rollins']  = np.minimum((df['A'] * ((df['B'] *  150)) ** df['C']), 15) * (p_d)    
    df['Pu1'] = (3 + (df['VerticalEffectiveStress[kPa]']/Sr) + (J/D)*df['Depth[m]']) * Sr * D
    df['Pu2'] = 9 * Sr * D
    df['Pu_wangReese'] = np.minimum(df['Pu1'], df['Pu2'])
    df['Pu'] = np.minimum(df['Pu_wangReese'], df['Pu_Rollins)'])
    return df


# Layer 1

In [5]:
Su = 15.0  # Shear strength
D = 2.0    # Diameter
J = 0.5    # Factor
target_depth = 12.0  # Target depth (m)
# target_value = 2194.83855  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

F1 = compute_f_integral(target_depth, api_clay_pu, Sigma_df, Su, D, J, d_adj=0)

print(f"F-integral is {F1} at {target_depth} m ")

F-integral is 2194.8385499999954 at 12.0 m 


# Layer 2

In [6]:
Su = 59.0  # Shear strength
D = 2.0    # Diameter
J = 0.5    # Factor
target_depth = 12.0  # Target depth (m)
target_value = F1  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

solution = fsolve(lambda x: compute_f_integral(target_depth, api_clay_pu, Sigma_df, Su, D, J, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_clay_pu,
    effSigma_df,
    Su=Su,
    D=D,
    J=J,
    d_adj=solution[0]
)

F1 = compute_f_integral(
    36,
    api_clay_pu,
    effSigma_df,
    Su=Su,
    D=D,
    J=J,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {36.0} m with d_adj = {solution[0]}: {F1}")

d_adj found: -7.58766620226229
F-integral at 12.0 m with d_adj = -7.58766620226229: 2194.8385500000863
F-integral at 36.0 m with d_adj = -7.58766620226229: 25014.262455893888


# Layer 3

In [7]:
phi = 37.0
D = 2.0
target_depth = 36.0  # Target depth (m)
target_value = F1  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

solution = fsolve(lambda x: compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_sand_pu,
    Sigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)

F1 = compute_f_integral(
    52.5,
    api_sand_pu,
    Sigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {52.5} m with d_adj = {solution[0]}: {F1}")

d_adj found: -29.41289609926034
F-integral at 36.0 m with d_adj = -29.41289609926034: 25014.26245589391
F-integral at 52.5 m with d_adj = -29.41289609926034: 315331.0895061922


In [8]:
phi = 40
D = 2.0
target_depth = 52.5  # Target depth (m)
target_value = F1  # Target F-integral value (kPa·m)
Sigma_df = effSigma_df.copy(deep=True)

print(f"target F1 = {target_value}")

solution = fsolve(lambda x: compute_f_integral(target_depth, api_sand_pu, Sigma_df, phi, D, d_adj = x) - target_value, 
    x0=0.0,)

print(f"d_adj found: {solution[0]}")

F0 = compute_f_integral(
    target_depth,
    api_sand_pu,
    Sigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)

F1 = compute_f_integral(
    60.0,
    api_sand_pu,
    Sigma_df,
    phi=phi,
    D=D,
    d_adj=solution[0]
)


print(f"F-integral at {target_depth} m with d_adj = {solution[0]}: {F0}")
print(f"F-integral at {52.5} m with d_adj = {solution[0]}: {F1}")

target F1 = 315331.0895061922
d_adj found: -32.66008412623744
F-integral at 52.5 m with d_adj = -32.66008412623744: 315331.08950619254
F-integral at 52.5 m with d_adj = -32.66008412623744: 657666.5122218326


# Layer 1

In [9]:
#Layer 1

# Create a deep copy
interpolated_df = effSigma_df.copy(deep=True)

# Define the adjustment value
d_adj = 0.0

# Add a new column for equivalent depth with a condition
interpolated_df['EquivalentDepth[m]'] = np.maximum(interpolated_df['Depth[m]'] + d_adj, 0)

# Define the inputs
Su = 15.0
D = 2.0
J = 0.5

# Compute Pu_1 and add as a new column
interpolated_df['Pu_1'] = (
    (3 * Su + interpolated_df['VerticalEffectiveStress[kPa]']) * D +
    J * Su * interpolated_df['EquivalentDepth[m]']
)

Pu_2 = 9*Su*D

interpolated_df['Pu'] = np.minimum(interpolated_df['Pu_1'], Pu_2)

# Initialize the F-integral column with zeros
interpolated_df['F-integral'] = 0.0

# Compute F-integral iteratively using the trapezoidal rule
for i in range(1, len(interpolated_df)):
    # Current and previous Pu values
    Pu_current = interpolated_df.loc[i, 'Pu']
    Pu_previous = interpolated_df.loc[i - 1, 'Pu']
    
    # Current and previous depth values
    Depth_current = interpolated_df.loc[i, 'EquivalentDepth[m]']
    Depth_previous = interpolated_df.loc[i - 1, 'EquivalentDepth[m]']
    
    # Trapezoidal integration for the current row
    delta_F = 0.5 * (Pu_current + Pu_previous) * (Depth_current - Depth_previous)
    
    # Add to the cumulative F-integral value from the previous row
    interpolated_df.loc[i, 'F-integral'] = interpolated_df.loc[i - 1, 'F-integral'] + delta_F

# Display the resulting DataFrame
interpolated_df



,Depth[m],effUnitWeight[kN/m3],VerticalEffectiveStress[kPa],EquivalentDepth[m],Pu_1,Pu,F-integral
0,0.00,4.0,0.00,0.00,90.000,90.000,0.000000
1,0.01,4.0,0.04,0.01,90.155,90.155,0.900775
2,0.02,4.0,0.08,0.02,90.310,90.310,1.803100
3,0.03,4.0,0.12,0.03,90.465,90.465,2.706975
4,0.04,4.0,0.16,0.04,90.620,90.620,3.612400
...,...,...,...,...,...,...,...
5996,59.96,10.0,422.60,59.96,1384.900,270.000,15144.038550
5997,59.97,10.0,422.70,59.97,1385.175,270.000,15146.738550
5998,59.98,10.0,422.80,59.98,1385.450,270.000,15149.438550
5999,59.99,10.0,422.90,59.99,1385.725,270.000,15152.138550


In [10]:
F_depth = 12.0
F1 = interpolated_df.loc[interpolated_df['Depth[m]'] == F_depth, 'F-integral'].values
print(f"F-integral is {F1[0]} at {F_depth} ")


F-integral is 2194.8385499999954 at 12.0 


# Layer 2

In [11]:
def api_clay_pu(df, d_adj, Su, D, J):
    d_adj = np.asscalar(d_adj) if isinstance(d_adj, np.ndarray) else d_adj  # Ensure scalar value
    df = df.copy(deep=True)
    df['EquivalentDepth[m]'] = np.maximum(df['Depth[m]'] + d_adj, 0)
    df['Pu_1'] = (3 * Su + df['VerticalEffectiveStress[kPa]']) * D + J * Su * df['EquivalentDepth[m]']
    df['Pu_2'] = 9 * Su * D
    df['Pu'] = np.minimum(df['Pu_1'], df['Pu_2'] )
    return df

In [12]:
#Layer 2

# Create a deep copy
interpolated_df = effSigma_df.copy(deep=True)
api_clay_pu(interpolated_df, 
            d_adj = 0.0,
            Su = 59.0,  # Shear strength
            D = 2.0,   # Diameter
            J = 0.5 )   # Factor)

,Depth[m],effUnitWeight[kN/m3],VerticalEffectiveStress[kPa],EquivalentDepth[m],Pu_1,Pu_2,Pu
0,0.00,4.0,0.00,0.00,354.000,1062.0,354.000
1,0.01,4.0,0.04,0.01,354.375,1062.0,354.375
2,0.02,4.0,0.08,0.02,354.750,1062.0,354.750
3,0.03,4.0,0.12,0.03,355.125,1062.0,355.125
4,0.04,4.0,0.16,0.04,355.500,1062.0,355.500
...,...,...,...,...,...,...,...
5996,59.96,10.0,422.60,59.96,2968.020,1062.0,1062.000
5997,59.97,10.0,422.70,59.97,2968.515,1062.0,1062.000
5998,59.98,10.0,422.80,59.98,2969.010,1062.0,1062.000
5999,59.99,10.0,422.90,59.99,2969.505,1062.0,1062.000


In [13]:
from scipy.optimize import fsolve
interpolated_df = effSigma_df.copy(deep=True)
# Function to compute F-integral at a target depth
def compute_f_integral(d_adj, Su, D, J, target_depth,):
    # Update the EquivalentDepth[m] column based on d_adj
    interpolated_df['EquivalentDepth[m]'] = np.maximum(interpolated_df['Depth[m]'] + d_adj, 0)
    
    # Recalculate Pu_1 and Pu
    interpolated_df['Pu_1'] = (
        (3 * Su + interpolated_df['VerticalEffectiveStress[kPa]']) * D +
        J * Su * interpolated_df['EquivalentDepth[m]']
    )
    Pu_2 = 9 * Su * D
    interpolated_df['Pu'] = np.minimum(interpolated_df['Pu_1'], Pu_2)
    
    # Reinitialize the F-integral column
    interpolated_df['F-integral'] = 0.0
    
    # Compute F-integral using the trapezoidal rule
    for i in range(1, len(interpolated_df)):
        Pu_current = interpolated_df.loc[i, 'Pu']
        Pu_previous = interpolated_df.loc[i - 1, 'Pu']
        Depth_current = interpolated_df.loc[i, 'EquivalentDepth[m]']
        Depth_previous = interpolated_df.loc[i - 1, 'EquivalentDepth[m]']
        delta_F = 0.5 * (Pu_current + Pu_previous) * (Depth_current - Depth_previous)
        interpolated_df.loc[i, 'F-integral'] = interpolated_df.loc[i - 1, 'F-integral'] + delta_F
    
    # Return the F-integral value at the target depth
    f_integral_at_target = interpolated_df.loc[interpolated_df['Depth[m]'] == target_depth, 'F-integral'].values
    return f_integral_at_target[0] if len(f_integral_at_target) > 0 else None
# Define the objective function for fsolve
def objective_fsolve(d_adj, Su, D, J, target_depth, target_value):
    return compute_f_integral(d_adj, Su, D, J, target_depth) - target_value
# Parameters
Su = 59.0  # Shear strength
D = 2.0    # Diameter
J = 0.5    # Factor
target_depth = 12.0  # Target depth (m)
target_value = 2194.8385499999954  # Target F-integral value (kPa·m)
initial_guess = 0.0
# Solve using fsolve
solution = fsolve(objective_fsolve, x0=initial_guess, args=(Su, D, J, target_depth, target_value))

# Verify the result
if np.isclose(objective_fsolve(solution[0], Su, D, J, target_depth, target_value), 0, atol=1e-6):
    print(f"d_adj found: {solution[0]}")
    print(f"F-integral at {target_depth}m with d_adj = {solution[0]}: {compute_f_integral(solution[0], Su, D, J, target_depth)}")
else:
    print("Failed to converge to a valid solution.")



d_adj found: -7.58766620226229
F-integral at 12.0m with d_adj = -7.58766620226229: 2194.8385500000863


In [14]:
from scipy.optimize import fsolve
import numpy as np
import pandas as pd

# Generalized function to compute F-integral
def compute_f_integral(target_depth, func, *args, **kwargs):
    df = func(*args, **kwargs)  # Apply the transformation function
    if 'Pu' not in df.columns or 'EquivalentDepth[m]' not in df.columns:
        raise ValueError("The input function must output a DataFrame with 'Pu' and 'EquivalentDepth[m]' columns.")
    df['F-integral'] = 0.0
    for i in range(1, len(df)):
        Pu_current = df.loc[i, 'Pu']
        Pu_previous = df.loc[i - 1, 'Pu']
        Depth_current = df.loc[i, 'EquivalentDepth[m]']
        Depth_previous = df.loc[i - 1, 'EquivalentDepth[m]']
        delta_F = 0.5 * (Pu_current + Pu_previous) * (Depth_current - Depth_previous)
        df.loc[i, 'F-integral'] = df.loc[i - 1, 'F-integral'] + delta_F
    f_integral_at_target = df.loc[df['Depth[m]'] == target_depth, 'F-integral'].values
    return f_integral_at_target[0] if len(f_integral_at_target) > 0 else None

# Example transformation function
def api_clay_pu(df, d_adj, Su, D, J):
    df = df.copy(deep=True)
    df['EquivalentDepth[m]'] = np.maximum(df['Depth[m]'] + d_adj, 0)
    Pu_1 = (3 * Su + df['VerticalEffectiveStress[kPa]']) * D + J * Su * df['EquivalentDepth[m]']
    Pu_2 = 9 * Su * D
    df['Pu'] = np.minimum(Pu_1, Pu_2)
    return df

# Objective function for fsolve
def objective_fsolve(d_adj, target_depth, target_value, func, *args, **kwargs):
    return compute_f_integral(target_depth, func, d_adj, *args, **kwargs) - target_value

# Example input DataFrame
effSigma_df = pd.DataFrame({
    'Depth[m]': [0, 12, 12.01, 36, 36.01, 52.5, 52.51, 60],
    'VerticalEffectiveStress[kPa]': [0, 48, 49, 252, 253, 420, 421, 480]  # Example data
})

# Parameters
Su = 59.0  # Shear strength
D = 2.0    # Diameter
J = 0.5    # Factor
target_depth = 12.0  # Target depth (m)
target_value = 2194.83855  # Target F-integral value (kPa·m)

# Initial guess for d_adj
initial_guess = 0.0

# Solve using fsolve
solution = fsolve(
    objective_fsolve,
    x0=initial_guess,
    args=(target_depth, target_value, api_clay_pu, effSigma_df, Su, D, J)
)

# Print results
f_integral_at_solution = compute_f_integral(
    target_depth,
    api_clay_pu,
    effSigma_df,
    d_adj=solution[0],
    Su=Su,
    D=D,
    J=J
)

print(f"d_adj found: {solution[0]}")
print(f"F-integral at {target_depth}m with d_adj = {solution[0]}: {f_integral_at_solution}")


TypeError: copy() got an unexpected keyword argument 'deep'

In [ ]:
#Layer 1

# Define the adjustment value
d_adj = solution[0]

# Add a new column for equivalent depth with a condition
interpolated_df['EquivalentDepth[m]'] = np.maximum(interpolated_df['Depth[m]'] + d_adj, 0)

# Define the inputs
Su = 59.0  # Shear strength
D = 2.0    # Diameter
J = 0.5    # Factor

# Compute Pu_1 and add as a new column
interpolated_df['Pu_1'] = (
    (3 * Su + interpolated_df['VerticalEffectiveStress[kPa]']) * D +
    J * Su * interpolated_df['EquivalentDepth[m]']
)

Pu_2 = 9*Su*D

interpolated_df['Pu'] = np.minimum(interpolated_df['Pu_1'], Pu_2)

# Initialize the F-integral column with zeros
interpolated_df['F-integral'] = 0.0

# Compute F-integral iteratively using the trapezoidal rule
for i in range(1, len(interpolated_df)):
    # Current and previous Pu values
    Pu_current = interpolated_df.loc[i, 'Pu']
    Pu_previous = interpolated_df.loc[i - 1, 'Pu']
    
    # Current and previous depth values
    Depth_current = interpolated_df.loc[i, 'EquivalentDepth[m]']
    Depth_previous = interpolated_df.loc[i - 1, 'EquivalentDepth[m]']
    
    # Trapezoidal integration for the current row
    delta_F = 0.5 * (Pu_current + Pu_previous) * (Depth_current - Depth_previous)
    
    # Add to the cumulative F-integral value from the previous row
    interpolated_df.loc[i, 'F-integral'] = interpolated_df.loc[i - 1, 'F-integral'] + delta_F

# Display the resulting DataFrame
interpolated_df



In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(interpolated_df)